In [1]:
# from openai import OpenAI
# from dotenv import load_dotenv
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset

os.environ["PATH"] += os.pathsep + r"C:\ProgramData\chocolatey\lib\ffmpeg\tools\ffmpeg\bin"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = "../audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])

print(result)


c:\Users\cryst\Desktop\LLM_Practic\AI_Practic\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language 

{'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다. GPT API에 대해서 생소하신 분들도 있을 텐데 우리가 잘 알고 있는 ChatGPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거에요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 체포 프로그램을 만드는게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 이야기를 할 거고요. 그 예제로 예제는 여러가지가 될 수 있는데 여기서 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 그래서 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.16, 10.0), 'text': ' GPT API에 대해서 생소하신 분들도 있을 텐데'}, {'timestamp': (11.0, 17.0), 'text': ' 우리가 잘 알고 있는 ChatGPT 기능을 이용해서'}, {'timestamp': (17.0, 20.0), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서'}, {'timestamp': (20.0, 22.0), 'text': ' 이야기할 거에요.'}, {'timestamp': (22.0, 24.0), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (24.0, 27.48), 'text': ' 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면'}, {'timestamp': (27.48, 29.58), 'text': ' GPT로 명확한 미션을 

In [2]:
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다.
1,7.16,10.00,GPT API에 대해서 생소하신 분들도 있을 텐데
2,11.00,17.00,우리가 잘 알고 있는 ChatGPT 기능을 이용해서
3,17.00,20.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서
4,20.00,22.00,이야기할 거에요.
5,22.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
6,24.00,27.48,그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
7,27.48,29.58,GPT로 명확한 미션을 달성하는
8,29.58,31.62,체포 프로그램을 만드는게 사실
9,31.62,33.00,쉽지는 않은데
